# Check Data Availability (RS)
* StelllarAlgo Data Science
* Ryan Kazmerik & Nakisa Rad
* May 13, 2022

In [9]:
import boto3 
import pandas as pd
import psycopg2

from shared_utilities import helpers

pd.options.display.max_columns = 100
pd.options.display.max_rows = 100

### Create a connection to RedShift PROD:

In [10]:
if not boto3.DEFAULT_SESSION:
    ! aws sso login --profile Stellaralgo-DataScienceAdmin
    session = boto3.setup_default_session(profile_name='Stellaralgo-DataScienceAdmin')

In [21]:
CLUSTER = "prod-app-elbu"
DBNAME = "stlrechladmirals"
LKUPCLIENTID = 16

CNXN = helpers.get_redshift_connection(CLUSTER, DBNAME)

### Run the stored proc to get retention training and inference datasets:

In [22]:
sql=f"""
    CALL ds.getretentionmodeldata(16, 2010, 2022, 'rkcursor')
"""

cur = CNXN.cursor()

try:
    cur.execute(sql)

    named_cursor = CNXN.cursor('rkcursor')
    data = named_cursor.fetchall()

    cols = [row[0] for row in named_cursor.description]
    df = pd.DataFrame(data=data, columns=cols)

    CNXN.commit()

except Exception as e:
    print(f"ERROR:, {e}")
    cur.execute("ROLLBACK")

finally:
    cur.close()

### Check how much data we have for each season:

In [24]:
print(f"DATA AVAILABILITY REPORT:")
print(f" > {CLUSTER} {DBNAME}")
df["year"].value_counts().sort_index(ascending=False)

DATA AVAILABILITY REPORT:
 > prod-app-elbu stlrechladmirals


Series([], Name: year, dtype: int64)

### Done